[![Lab Documentation and Solutions](https://img.shields.io/badge/Lab%20Documentation%20and%20Solutions-purple)](https://mongodb-developer.github.io/vector-search-lab/)

In [4]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AnyMessage
from langgraph.runtime import get_runtime
from dataclasses import dataclass
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel

class AgentResponse(BaseModel):
    # ensure strict compliance with the schema
    weather_conditions: str
    temperature: float
    address: str

@dataclass
class ContextSchema:
    user_name: str

# Load environment variables from .env file
load_dotenv()

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

def get_address(user: str) -> str:
    """Get users address"""
    return f"{user} lives at London, Elephant and Castle"

def prompt(state: AgentState) -> list[AnyMessage]:
    runtime = get_runtime(ContextSchema)
    system_msg = f"You are a helpful assistant. Address the user as {runtime.context.user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

agent = create_react_agent(
    model=init_chat_model(
        # model="openai:gpt-5",
        model="openai:gpt-4o-mini",
        # api_key=os.getenv("OPENAI_API_KEY"),
        temperature=0.0,
        # max_tokens=1000,
    ),
    # the AI model will be queried to choose which tool to use based on the user's request
    tools=[get_weather, get_address],
    
    prompt=prompt,
    context_schema=ContextSchema,
    
    # the agent will save its state in memory
    checkpointer=InMemorySaver(),
    # checkpointer=MongoDBSaver(
    #     client=get_client().client,
    #     checkpointCollectionName="agent_state",
    #     db_name="playground"
    # ),
    
    # Structured output requires an additional call to the LLM to format the response according to the schema.
    response_format=AgentResponse  
)


In [5]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf?"}]},
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"}
)

{'messages': [HumanMessage(content='what is the weather in sf?', additional_kwargs={}, response_metadata={}, id='7ae74b78-c08a-47f6-9e25-5a82609f5638'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OURNIpc9nVzTfQxrwkYLaMHQ', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3QWGAgBEJChGksb1oW8PtFHAdGME', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--93b3cdf7-fd1b-4399-aa4e-5e2e14ce215c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'},

In [3]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "Where does she live?"}]},
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"}
)

{'messages': [HumanMessage(content='what is the weather in sf?', additional_kwargs={}, response_metadata={}, id='8c81784c-cc07-4430-b07b-bbdcba77a31f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LI9YWH12VF7UcziQR6fITTHU', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3QLNFbLT9UGSdHU3vkXxsOizMdfW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e03854cf-69dd-4fad-9fa4-da938f494271-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'},

In [4]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "Where does she live, and is it raining there?"}]},
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"}
)

{'messages': [HumanMessage(content='what is the weather in sf?', additional_kwargs={}, response_metadata={}, id='8c81784c-cc07-4430-b07b-bbdcba77a31f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LI9YWH12VF7UcziQR6fITTHU', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3QLNFbLT9UGSdHU3vkXxsOizMdfW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e03854cf-69dd-4fad-9fa4-da938f494271-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'},

In [5]:
# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "What did I say?"}]},
    context=ContextSchema(user_name="Ricardo Villanueva"),
    config={"thread_id": "123"}
)

{'messages': [HumanMessage(content='what is the weather in sf?', additional_kwargs={}, response_metadata={}, id='8c81784c-cc07-4430-b07b-bbdcba77a31f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LI9YWH12VF7UcziQR6fITTHU', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 84, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3QLNFbLT9UGSdHU3vkXxsOizMdfW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--e03854cf-69dd-4fad-9fa4-da938f494271-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'},

In [1]:

from src.examples.agent_with_mongodb_memory import invoke_agent

invoke_agent("What is the weather in London?", "John", "123")

/Users/ricardo.villanueva/miniconda3/envs/ai-agent-with-mongodb/lib/python3.13/site-packages/pymongo/common.py:844: UserWarning: Wire protocol compression with snappy is not available. You must install the python-snappy module for snappy support.
  value = validator(option, value)


{'messages': [HumanMessage(content='What is the weather in London?', additional_kwargs={}, response_metadata={}, id='629f5486-601a-47c1-b98c-af137b6b8873'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lh33xC0RUtO5emzjzY03AZxR', 'function': {'arguments': '{"city":"London"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 165, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3QV2IcA6E4qQ9QR2UU3eSAkVFRWY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--17fcee91-a53a-4278-82c8-4aea49b53d7e-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'London'}, 'id': 'call_lh33xC0

In [7]:

from src.examples.agent_with_mongodb_memory import invoke_agent

invoke_agent("What is the weather in London?", "John", "123")

{'messages': [HumanMessage(content='What is the weather in London?', additional_kwargs={}, response_metadata={}, id='520441fc-686a-4f11-92ee-6cf967f82f9a'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8fMmUCb0u3kDIfkEJfJKlaYb', 'function': {'arguments': '{"city":"London"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 165, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-C3QZShExu4mqV3JR5ZgRiTtoIpC3A', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--c3e26db6-0aa0-4728-928b-7d12dd781195-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'London'}, 'id': 'call_8fMmUCb